In [83]:
# playing around with lstms and text

from __future__ import print_function
import tensorflow as tf
import numpy as np
import os
import string
from six.moves.urllib.request import urlretrieve
import re
import random

In [84]:
# url = 'http://constitution.org/'
url = 'http://www.gutenberg.org/cache/epub/18500/'
def download(filename):
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  return filename

filename = download('pg18500.txt')
# filename = download('usdeclar.txt')
print(filename)

pg18500.txt


In [85]:
def read_data(filename):
    with open(filename, 'rb') as f:
        data = tf.compat.as_str(f.read())
    return data

def clean(arr):
    str = ''.join([c for c in arr if c.isalpha() or c == ' ' or c == '\n'])
    str = re.sub(r"[ \n]{2,}", " ", str)
    str = re.sub(r"[\n]", " ", str)
    # remove empty tokens
    cleaned = [tok for tok in str.split(' ') if tok]
    return ' '.join(cleaned), cleaned

text, text_list = clean(read_data(filename))
#text = read_data(filename)
print('Data size %d' % len(text))

Data size 1847013


In [86]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:105])
print(valid_size, valid_text[:105])

1846013 hroughout the book No change has been made in this The relative indentations of Poems Epitaphs and Songs 
1000 The Project Gutenberg EBook of The Complete Works of Robert Burns Containing his Poems Songs and Correspo


In [87]:
print(len(text_list))

348008


In [88]:
# find map word -> id

words = dict()
ids = dict()
index = 0
for word in text_list:
    if word not in words:
        words[word] = index
        ids[index] = word
        index = index + 1
# print(words['Gutenberg'])
# print(words['Works'])
print(ids[0])

vocab_size = len(words)
print(vocab_size)

The
29983


In [89]:
# generate batches

batch_size=15
num_unrollings=6

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocab_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, words[self._text[self._cursor[b]]]] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  words back into its (most likely) character representation."""
  return [ids[c] for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  print(np.array(batches).shape)
  print('vocab size:', vocab_size)
  for b in batches:
    s = [' '.join(x) for x in zip(s, characters(b))]
  return s


In [90]:
batch_tester = BatchGenerator(text_list, batch_size, num_unrollings)
batches = batch_tester.next()
batches_np = np.array(batches)
print(batches_np.shape)
print(np.count_nonzero(batches_np))



(7, 15, 29983)
105


In [91]:
new_batches = batch_tester.next()
new_np = np.array(new_batches)

print(batches2string(new_batches))

print(new_np.shape)
print(np.count_nonzero(new_np))

# verify batches are different
print(np.array_equal(batches_np, new_np))


(7, 15, 29983)
vocab size: 29983
[' Complete Works of Robert Burns Containing his', ' that by his skill in rearing sheep', ' unison between the pair An made the', ' your duty duly morn and night Lest', ' was first printed by Stewart in The', ' warmer heart death neer made cold III', ' chair shall fa He is the king', ' liberty and independence which I threw into', ' dear Sir Yours ROBT BURNESS FOOTNOTES Footnote', ' rhymes The first of my poetic offspring', ' was talked of We got a song', ' the one side and my fiddlecase on', ' other In the honest enthusiasm with which', ' me long ago but over and above', ' they disguise as like wretched old age']
(7, 15, 29983)
105
False


In [53]:
# single layer RNN

batch_size = 10
num_unrollings = 12
num_cells = num_unrollings - 1
# cell output size
state_size = 30

tf.reset_default_graph()

# trying without embeddings (only word ids)

# 1-hot
# inputs = list length num_inputs of [batch_size, vocab_size]
inputs = [tf.placeholder(tf.float32, shape=[None, vocab_size]) for _ in range(num_cells)]
labels = [tf.placeholder(tf.float32, shape=[None, vocab_size]) for _ in range(num_cells)]

# 2-tuple initial state
state = tf.placeholder(tf.float32, shape=[None, state_size])
hidden_state = tf.placeholder(tf.float32, shape=[None, state_size])

cell = tf.contrib.rnn.BasicLSTMCell(state_size, state_is_tuple=True)

# outputs list of [batch_size, output_size]
outputs, final_state = tf.contrib.rnn.static_rnn(cell, inputs, (state, hidden_state), scope="tf.contrib.rnn.BasicLSTMCell")

stddev1 = np.sqrt(2.0 / state_size * vocab_size)
w = tf.Variable(tf.truncated_normal([state_size, vocab_size], stddev=stddev1))
stddev2 = np.sqrt(2.0 / vocab_size)
b = tf.Variable(tf.truncated_normal([vocab_size], stddev=stddev2))

losses = []
predictions = []
for i in range(num_cells):
    logits = tf.matmul(outputs[i], w) + b
    predictions.append(tf.argmax(logits, 1))
    losses.append(tf.nn.softmax_cross_entropy_with_logits(labels=labels[i], logits=logits))

# losses = [tf.nn.softmax_cross_entropy_with_logits(labels=labels[i], 
#             logits=tf.matmul(outputs[i], w) + b) for i in range(num_cells)]

loss = tf.reduce_mean(losses)
optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [54]:
# indices of words i.e. [batch_size]
def get_words(indices):
    l = []
    for i in indices:
        l.append(ids[i])
    return l

# batch_size = 10
# num_unrollings = 12
# num_cells = num_unrollings - 1

# train
num_iterations = 30000

sess = tf.Session()
sess.run(tf.global_variables_initializer())

generator = BatchGenerator(text_list, batch_size, num_unrollings)
for i in range(num_iterations):
    batches = generator.next()
    feed_dict = dict()
    for j in range(num_cells):
        feed_dict[inputs[j]] = batches[j]
        feed_dict[labels[j]] = batches[j + 1]
    feed_dict[state] = np.zeros((batch_size, state_size))
    feed_dict[hidden_state] = np.zeros((batch_size, state_size))
    
    _, l, pred, labs, input_np = sess.run([optimizer, loss, predictions, labels, inputs], feed_dict=feed_dict)
    if i % 10 == 0 and i != 0:
        print('Loss: %f' % l)
    if i % 20 == 0 and i != 0:
        rand_ind = random.randint(0, batch_size - 1)
        input_np = np.array(input_np)
        labs = np.array(labs)
        pred = np.array(pred)
        print(input_np.shape)
        print('Input:', ' '.join(get_words(np.argmax(input_np[:, rand_ind], 1))))
        print('Labels:', ' '.join(get_words(np.argmax(labs[:, rand_ind], 1))))
        print('Predict:', ' '.join(get_words(pred[:, rand_ind])))

Loss: 10.520134
Loss: 10.520571
(11, 10, 29983)
Input: money paid him in drink and paid him in verse With
Labels: paid him in drink and paid him in verse With Pegasus
Predict: cutting billetdoux excuses chasd namesake portraits portraits excuses stripe Ghaist Sonnet
Loss: 10.296631
Loss: 10.362860
(11, 10, 29983)
Input: very devil He though two years younger has completely mastered his
Labels: devil He though two years younger has completely mastered his brother
Predict: PUBLIC fiddlers Wrongs stripe attour poverty Gully censure Gully hearthstone advertised
Loss: 10.260087
Loss: 10.286766
(11, 10, 29983)
Input: to my mou gied a sten For thrice I drew ane
Labels: my mou gied a sten For thrice I drew ane without
Predict: the shear shear stroll parched Vengeance parched Vengeance Symon Vengeance Gully
Loss: 10.162065
Loss: 10.269129
(11, 10, 29983)
Input: first order When I grow richer I will write to you
Labels: order When I grow richer I will write to you on
Predict: flowrreviving Arian

KeyboardInterrupt: 

In [39]:
# generate new stuff

num_gen = 50

# take 1st part of it
test_generator = BatchGenerator(text_list, 1, num_unrollings)
next_in = test_generator.next()
for i in range(num_gen):
    feed_dict = dict()
    for j in range(num_cells):
        feed_dict[inputs[j]] = next_in[j]
        # ignored
        feed_dict[labels[j]] = next_in[j]
    # clears previous state
    feed_dict[state] = np.zeros((1, state_size))
    feed_dict[hidden_state] = np.zeros((1, state_size))
    pred = sess.run(predictions, feed_dict=feed_dict)
    print(' '.join(get_words(np.array(pred).reshape((-1)))))
    next_in = np.zeros(shape=[num_cells, 1, vocab_size])
    for i in range(num_cells):
        next_in[i, 0, pred[i]] = 1.0

of the Adopted in Congress July The Unanimous Declaration of the
the people in Congress July The Unanimous Declaration of the Thirteen
most to the July The Unanimous Declaration of the Thirteen United
humble valuable Creator The Unanimous Declaration of the Thirteen United States
terms becomes certain laws laws of Thirteen Thirteen United States of
our our places form form nature United United States of America
most British unusual happiness as and and States of America in
humble Crown power encourage encourage endowed them them America in General
terms and all their their by guards guards created created Congress
our foreign cases high high repeated for for power equal their
most mercenaries full seas seas bear bear the giving independent this
humble to to to to be arms arms his convulsions totally
terms terms our redress redress formidable against against against within judiciary
our our repeated in in trade bodies us us He He
most British petitions petitions death all all all all ha

In [94]:
# trying multi layer RNN

batch_size = 10
num_unrollings = 12
num_cells = num_unrollings - 1
layers = 3
# cell output size
state_size = 30

tf.reset_default_graph()

# trying without embeddings (only word ids)

# 1-hot
# inputs = list length num_inputs of [batch_size, vocab_size]
inputs = [tf.placeholder(tf.float32, shape=[None, vocab_size]) for _ in range(num_cells)]
labels = [tf.placeholder(tf.float32, shape=[None, vocab_size]) for _ in range(num_cells)]

# init state = [layers, 2, None, state_size]
states = tf.placeholder(tf.float32, shape=[layers, 2, None, state_size])

# list of tensors
initial_state = [ (states[i][0], states[i][1]) for i in range(layers)]

cell = tf.contrib.rnn.LSTMCell(state_size)
cell = tf.contrib.rnn.MultiRNNCell([cell] * layers, state_is_tuple=True)

outputs, final_state = tf.contrib.rnn.static_rnn(cell, inputs, initial_state=initial_state)

stddev1 = np.sqrt(2.0 / state_size * vocab_size)
w = tf.Variable(tf.truncated_normal([state_size, vocab_size], stddev=stddev1))
stddev2 = np.sqrt(2.0 / vocab_size)
b = tf.Variable(tf.truncated_normal([vocab_size], stddev=stddev2))

losses = []
predictions = []
for i in range(num_cells):
    logits = tf.matmul(outputs[i], w) + b
    predictions.append(tf.argmax(logits, 1))
    losses.append(tf.nn.softmax_cross_entropy_with_logits(labels=labels[i], logits=logits))

# losses = [tf.nn.softmax_cross_entropy_with_logits(labels=labels[i], 
#             logits=tf.matmul(outputs[i], w) + b) for i in range(num_cells)]

loss = tf.reduce_mean(losses)
optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [97]:
# multi layer
# indices of words i.e. [batch_size]
def get_words(indices):
    l = []
    for i in indices:
        l.append(ids[i])
    return l

# batch_size = 10
# num_unrollings = 12
# num_cells = num_unrollings - 1

# train
num_iterations = 30000

sess = tf.Session()
sess.run(tf.global_variables_initializer())

generator = BatchGenerator(text_list, batch_size, num_unrollings)
for i in range(num_iterations):
    batches = generator.next()
    feed_dict = dict()
    for j in range(num_cells):
        feed_dict[inputs[j]] = batches[j]
        feed_dict[labels[j]] = batches[j + 1]
    feed_dict[states] = np.zeros((layers, 2, batch_size, state_size))
    
    _, l, pred, labs, input_np = sess.run([optimizer, loss, predictions, labels, inputs], feed_dict=feed_dict)
    if i % 25 == 0 and i != 0:
        print('Loss: %f' % l)
    if i % 50 == 0 and i != 0:
        rand_ind = random.randint(0, batch_size - 1)
        input_np = np.array(input_np)
        labs = np.array(labs)
        pred = np.array(pred)
        print(input_np.shape)
        print('Input:', ' '.join(get_words(np.argmax(input_np[:, rand_ind], 1))))
        print('Labels:', ' '.join(get_words(np.argmax(labs[:, rand_ind], 1))))
        print('Predict:', ' '.join(get_words(pred[:, rand_ind])))

Loss: 10.275769
Loss: 10.133775
(11, 10, 29983)
Input: illadvised people had uncoupled the merciless pack of the law at
Labels: people had uncoupled the merciless pack of the law at my
Predict: the spoonful spoonful spoonful spoonful spoonful spoonful spoonful spoonful spoonful MEN
Loss: 10.136617
Loss: 10.120363
(11, 10, 29983)
Input: Fingal The god of the bottle sends down from his hall
Labels: The god of the bottle sends down from his hall This
Predict: the spoonful spoonful spoonful spoonful spoonful spoonful spoonful spoonful spoonful spoonful
Loss: 10.116810
Loss: 10.119184
(11, 10, 29983)
Input: he was singing the tears down came Therell never be peace
Labels: was singing the tears down came Therell never be peace till
Predict: the the the the the the the MEN MEN MEN MEN
Loss: 10.076747
Loss: 10.014558
(11, 10, 29983)
Input: them with life and beauty His songs have all the beauties
Labels: with life and beauty His songs have all the beauties and
Predict: the the the the MEN MEN 

KeyboardInterrupt: 